In [2]:
import os

# folder_path = 'C:/Users/DuYih/Desktop/PVC/Single_Sample'

# output_file = open('output.csv', 'w')
# output_file.write('File,Line\n')  # 输出文件头

# for dirpath, dirnames, filenames in os.walk(folder_path):
#     for filename in filenames:
#         if filename == 'classification.txt':
#             filepath = os.path.join(dirpath, filename)
#             with open(filepath, 'r') as f:
#                 for line in f:
#                     if 'Verrucomicrobia' or 'Planctomycetales' or 'Chlamydiae' in line:
#                         output_file.write('{},{}'.format(filepath, line))
                        
# output_file.close()


# Verrucomicrobia in maps

In [3]:
def filter_samples(sample_data_path, keyword="Verrucomicrobia"):
    with open(sample_data_path, 'r', encoding='utf-8') as f:  # specify the encoding
        content = f.read()

    samples = content.split(">")[1:]  
    filtered_samples = [sample for sample in samples if keyword in sample]

    return filtered_samples


In [4]:
verrucomicrobia_samples = filter_samples('samples.info')

MemoryError: 

In [12]:
def filter_samples(sample_data_path, keyword="Verrucomicrobia"):
    with open(sample_data_path, 'r', encoding='utf-8') as f:
        sample = ""
        for line in f:
            if line.startswith(">"):
                if keyword in sample:
                    yield sample
                sample = line
            else:
                sample += line
        if keyword in sample:  # don't forget the last sample
            yield sample

# with open('verrucomicrobia_samples.txt', 'w') as f:
#     for sample in filter_samples('samples.info'):
#         f.write(">" + sample) 

with open('verrucomicrobia_samples.txt', 'w', encoding='utf-8') as f:
    for sample in filter_samples('samples.info'):
        f.write(sample) 


In [19]:
import requests
import xml.etree.ElementTree as ET

def get_biosample_data(biosample_accession):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=biosample&id={biosample_accession}&rettype=xml"
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        raise ValueError("Failed to fetch biosample data")

def get_bacterial_proportion(xml_content, bacterium_name):
    root = ET.fromstring(xml_content)
    bacterial_proportion = None
    for sample in root.findall('.//BioSample'):
        organism = sample.find('.//Organism')
        if organism.get('taxonomy_name') == bacterium_name:
            attributes = sample.findall('.//Attribute')
            for attribute in attributes:
                if attribute.get('attribute_name') == 'organism_percent':
                    bacterial_proportion = attribute.text
                    break
    return bacterial_proportion

# 用法示例
biosample_accession = '>ERS1882903'  # 替换为实际的 biosample accession 号码
bacterium_name = 'Verrucomicrobia'  # 替换为实际的细菌名称
xml_content = get_biosample_data(biosample_accession)
proportion = get_bacterial_proportion(xml_content, bacterium_name)
print(f"The proportion of {bacterium_name} in the sample is: {proportion}")


The proportion of Verrucomicrobia in the sample is: None
